In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("15"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Warehouse Woes

[Open Website](https://adventofcode.com/2024/day/15)

In [2]:
from common.matrix import Matrix, MatrixNavigator


# helper functions
def domain_from_input(input: InputReader) -> (Matrix, list):
    lines = input.lines_as_str()

    matrix = []
    instructions = []

    # building map
    building_map = True
    for line in lines:
        if building_map:
            # split on characters
            matrix.append(list(line))

            if len(line) == 0:
                building_map = False
        else:
            for next in line:
                instructions.append(next)

    return Matrix(matrix), instructions


matrix, instructions = domain_from_input(puzzle.example(0))
matrix.print()
print(instructions)

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

['<', 'v', 'v', '>', '^', '<', 'v', '^', '>', 'v', '>', '^', 'v', 'v', '^', 'v', '>', 'v', '<', '>', 'v', '^', 'v', '<', 'v', '<', '^', 'v', 'v', '<', '<', '<', '^', '>', '<', '<', '>', '<', '>', '>', 'v', '<', 'v', 'v', 'v', '<', '>', '^', 'v', '^', '>', '^', '<', '<', '<', '>', '<', '<', 'v', '<', '<', '<', 'v', '^', 'v', 'v', '^', 'v', '>', '^', 'v', 'v', 'v', '<', '<', '^', '>', '^', 'v', '^', '^', '>', '<', '<', '>', '>', '>', '<', '>', '^', '<', '<', '>', '<', '^', 'v', 'v', '^', '^', '<', '>', 'v', 'v', 'v', '<', '>', '>', '<', '^', '^', 'v', '>', '^', '>', 'v', 'v', '<', '>', 'v', '<', '<', '<', '<', 'v', '<', '^', 'v', '>', '^', '<', '^', '^', '>', '>', '>', '^', '<', 'v', '<', 'v', '>', '<', '>', 'v', 'v', '>', 'v', '^', 'v', '^', '<', '>', '>', '<', '>', '>', '>', '>', '<', '^', '^', '>', 'v', 'v', '>', 'v', '<', '^', '^', '^', '>', '>', 'v', '^', 'v', '^', '<', '^'

In [6]:
from common.matrix import Direction

# test case (part 1)
instructions_map = {
    ">": Direction.RIGHT,
    "<": Direction.LEFT,
    "^": Direction.UP,
    "v": Direction.DOWN
}


def part_1(reader: InputReader, debug: bool) -> int:
    matrix, instructions = domain_from_input(reader)

    if debug:
        matrix.print()

    # find start
    pointer = None
    for x, y, value in matrix:
        if value == "@":
            pointer = MatrixNavigator(matrix, x, y)
            break

    # run through instructions
    for instruction in instructions:
        direction = instructions_map[instruction]
        _, value = pointer.peek_value(direction)
        if value == ".":  # open
            pointer.set_value(".")
            pointer.move(direction)
            pointer.set_value("@")
        elif value == "O":  # wall
            # copy pointers
            candidate = pointer.copy()
            candidate.move(direction)
            scout = candidate.copy()
            done = False
            while not done:
                scout.move(direction)
                value = scout.get_value()
                if value == ".":
                    scout.set_value("O")
                    pointer.set_value(".")
                    pointer.move(direction)
                    pointer.set_value("@")
                    done = True
                if value == "#":  # hit wall, stop
                    done = True

    total = 0
    for x, y, value in matrix:
        if value == "O":
            total += y* 100 + x

    return total


result = part_1(puzzle.get_code_block(1), True)
display(result)
assert result == 2028

result = part_1(puzzle.get_code_block(0), True)
display(result)
assert result == 10092

########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########



2028

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########



10092

In [7]:
# real case (part 1)
result = part_1(puzzle.input(), False)
display(result)

1442192

In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        display(lines)
    return 0


result = part_2(puzzle.example(0), True)
display(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()